<a href="https://colab.research.google.com/github/eeyvee-0x4d/eeyvee-0x4d/blob/main/Notebook/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Clone repository</h1>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import pandas as pd
from google.colab import data_table
data_table.enable_dataframe_formatter()


df = pd.read_csv('/gdrive/MyDrive/vaccination_all_tweets.csv')

In [ ]:
df = df.drop(columns=['id', 'user_description', 'user_created', 'user_followers', 'user_friends', 'user_favourites', 'user_verified', 'hashtags', 'source', 'retweets', 'favorites', 'is_retweet'])
df.to_csv(path_or_buf='/content/tweets.csv', index=False)

In [ ]:
txtfile = open('/content/locations.txt', 'r')
locations = txtfile.read().splitlines()

df = pd.read_csv('/content/tweets.csv')
tweets = df[df['user_location'].isin(locations)]
tweets.to_csv(path_or_buf='/content/tweets.csv', index=False)

In [1]:
# Clone the entire repo.
!git clone -l -s https://github.com/eeyvee-0x4d/cs-thesis thesis
%cd thesis
!ls

Cloning into 'thesis'...
remote: Enumerating objects: 834, done.
remote: Counting objects: 100% (834/834), done.
remote: Compressing objects: 100% (599/599), done.
remote: Total 834 (delta 275), reused 690 (delta 203), pack-reused 0
Receiving objects: 100% (834/834), 101.46 MiB | 19.56 MiB/s, done.
Resolving deltas: 100% (275/275), done.
/content/thesis
client	Dataset  kaggle_tweets.csv  Notebook  README.md  server  StreamingAPI


In [2]:
import re
import pandas as pd

In [ ]:
df = pd.read_csv('/content/thesis/Dataset/Annotated-Compilation.csv', encoding='cp1252')

<h1>Text Preprocessing</h1>
<ul>
  <li>Import dataset</li>
  <li>Remove urls</li>
  <li>Remove special characters</li>
  <li>Convert text data to lowercase</li>
</ul>

In [3]:
df = pd.read_csv('/content/thesis/Dataset/trainingdata.csv') #read csv

df_shape = df.shape # (row, column)

# remove urls, remove special chars, conver to lowercase
for i in range(df_shape[0]):
  string = re.sub(r'http\S+', '', df.at[i, 'Text']).lower()
  df.at[i, 'Text'] = re.sub(r'\n', '', string)

<h1>Natural Language Toolkit NLTK</h1>
<p>
Nltk will be used to preprocess to corpus.
</p>

In [4]:
!pip install stopwordsiso

import nltk
import stopwordsiso

from nltk.stem import *
from nltk.corpus import stopwords
from nltk.util import ngrams

nltk.download("punkt")
nltk.download("stopwords")

     |████████████████████████████████| 73 kB 1.8 MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

<h1>Stemming words using Porter Stemmer</h1>
<p>
Apply porter stemmer to each tokens first then rebuild the tokens into sentence.
</p>
<hr>
<h1>Stop words removal</h1>
<p>
Remove stop words in english and tagalog.
</p>

In [5]:
corpus = df["Text"]

stemmer = PorterStemmer() # Porter Stemmer

stopwords_eng = set(stopwords.words('english')) # English stopwords
stopwords_tl  = set(stopwordsiso.stopwords('tl'))
filtered_sentence = []
filtered_sentence2 = []

for i in range(len(df['Text'])):
  document = df.loc[i, 'Text']
  tokens = nltk.word_tokenize(document)

  stemmed_tokens = [stemmer.stem(token) for token in tokens] # stem each words
  filtered_sentence = [token for token in stemmed_tokens if not token in stopwords_eng] # remove english stopwords
  filtered_sentence2 = [token for token in filtered_sentence if not token in stopwords_tl] #remove tagalog stopwords

  document = " ".join(filtered_sentence2)
  df.loc[i, 'Text'] = document

# Create train data and test data

Create train data and test data using 2:1 ration. Use `train_test_split` for this.


---


# Create n-grams from 1-4 ?

Create n-grams from 1 to 4 for exprementational purposes. Use params `ngram_range=(1,1)`, `ngram_range=(1,2)`, `ngram_range=(1,3)`, `ngram_range=(1,4)` in `TfidfVectorizer(ngram_range=(1,1))`.
Default is `ngram_range=(1,1)`



---


# Perform TF-IDF to the corpus


In [55]:
!pip install imbalanced-learn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['Text'], df['Sentiment']) # split dataset into train set and test set.
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
#x = text
#y = labels

(738,)
(738,)
(246,)
(246,)


In [62]:
x = df['Text']
y = df['Sentiment']
y.value_counts()

1    752
0    232
Name: Sentiment, dtype: int64

In [91]:
# 1-gram
vectorizer = TfidfVectorizer(ngram_range=(1,4)) # Initialize vectorizer
x_train_features = vectorizer.fit_transform(x)
# x_test_features = vectorizer.transform(x_test)

In [92]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')
x_oversampled, y_oversampled = oversample.fit_resample(x_train_features, y)

## Using Unvalidated data
##### unigram

|index|param\_alpha|mean\_test_accuracy|mean\_test_f1|mean\_test_precision|mean\_test_recall|
|---|---|---|---|---|---|
|0|1|0\.8433725005153578|0\.9067234670711187|0\.837956268254642|0\.9893333333333333|
|1|0\.1|0\.8983405483405482|0\.9327723203061895|0\.9406336889242061|0\.9267719298245615|
|2|0\.01|0\.899340342197485|0\.9327440790656707|0\.9458449249506972|0\.9214385964912282|
|3|0\.001|0\.8922180993609565|0\.9281631520729385|0\.9354407746148251|0\.9227543859649122|
|4|0\.0001|0\.886105957534529|0\.9246591113401206|0\.9266756053486711|0\.9240877192982456|
|5|1e-05|0\.8820449391877963|0\.9221884015807955|0\.9218514052853953|0\.9240877192982456|

##### Oversampled
|index|param\_alpha|mean\_test_accuracy|mean\_test_f1|mean\_test_precision|mean\_test_recall|
|---|---|---|---|---|---|
|0|1|0\.9521324503311259|0\.9499405203442693|0\.9794461997362995|0\.9241052631578949|
|1|0\.1|0\.9554525386313465|0\.9533241530524375|0\.9876281613123717|0\.9227719298245616|
|2|0\.01|0\.9554392935982341|0\.9531634283173396|0\.9875497145391661|0\.9227543859649122|
|3|0\.001|0\.9547770419426049|0\.9525383134065105|0\.9862559550262284|0\.9227543859649122|
|4|0\.0001|0\.9547814569536426|0\.9527225059628208|0\.9849743419025101|0\.9240877192982456|
|5|1e-05|0\.9554481236203092|0\.9535394994268731|0\.9849743419025101|0\.9254210526315789|

##### bigram
|index|param\_alpha|mean\_test_accuracy|mean\_test_f1|mean\_test_precision|mean\_test_recall|
|---|---|---|---|---|---|
|0|1|0\.8058338486909916|0\.8870131045503321|0\.8007805586879021|0\.9946666666666667|
|1|0\.1|0\.883044733044733|0\.9198779621790887|0\.9493961578963102|0\.8947894736842106|
|2|0\.01|0\.8698309626881056|0\.9078215343401377|0\.9654297527327534|0\.8615087719298247|
|3|0\.001|0\.8769532055246341|0\.9145601253543392|0\.9546724465501502|0\.8814736842105264|
|4|0\.0001|0\.8789837146980004|0\.9172551137078895|0\.9442490419115984|0\.8948070175438596|
|5|1e-05|0\.8769532055246341|0\.916725993662902|0\.9354774278902702|0\.9014561403508772|

##### Oversampled
|index|param\_alpha|mean\_test_accuracy|mean\_test_f1|mean\_test_precision|mean\_test_recall|
|---|---|---|---|---|---|
|0|1|0\.9594437086092716|0\.9576599408423313|0\.9808222788865244|0\.9374035087719298|
|1|0\.1|0\.9694172185430464|0\.9681659122380083|0\.9896391279952923|0\.9494210526315789|
|2|0\.01|0\.9660794701986755|0\.9644538888042138|0\.9922077922077921|0\.9400877192982456|
|3|0\.001|0\.9620971302428256|0\.9603240272426181|0\.992034632034632|0\.9320877192982457|
|4|0\.0001|0\.9627637969094923|0\.9610424180472157|0\.992034632034632|0\.9334210526315789|
|5|1e-05|0\.9627637969094923|0\.9610424180472157|0\.992034632034632|0\.9334210526315789|

##### trigram
|index|param\_alpha|mean\_test_accuracy|mean\_test_f1|mean\_test_precision|mean\_test_recall|
|---|---|---|---|---|---|
|0|1|0\.794671201814059|0\.8814627714049127|0\.7899591928578678|0\.9973333333333333|
|1|0\.1|0\.8627190270047412|0\.9032874487637794|0\.9588054154178642|0\.8574736842105264|
|2|0\.01|0\.826149247577819|0\.869656355174962|0\.9822107527437753|0\.7869298245614035|
|3|0\.001|0\.8484951556380128|0\.8902175463398383|0\.9741420880560527|0\.8242456140350878|
|4|0\.0001|0\.8556380127808699|0\.8977069801866697|0\.9570132861241067|0\.849561403508772|
|5|1e-05|0\.8566687280972995|0\.9002367045804576|0\.945278833891248|0\.8628771929824561|

##### Oversampled
|index|param\_alpha|mean\_test_accuracy|mean\_test_f1|mean\_test_precision|mean\_test_recall|
|---|---|---|---|---|---|
|0|1|0\.9634216335540838|0\.9613803560865797|0\.9918488102049746|0\.9347368421052632|
|1|0\.1|0\.9714083885209714|0\.9702352111718398|0\.9906287329848974|0\.9520701754385964|
|2|0\.01|0\.9694172185430464|0\.9682717560035832|0\.9905726837850125|0\.9480877192982454|
|3|0\.001|0\.9674172185430464|0\.9659486883127437|0\.9918173310296599|0\.9427543859649123|
|4|0\.0001|0\.965421633554084|0\.9638334137047426|0\.9917792792792792|0\.9387719298245614|
|5|1e-05|0\.9647549668874174|0\.9631150229001448|0\.9917792792792792|0\.9374385964912282|

##### 4-gram
|index|param\_alpha|mean\_test_accuracy|mean\_test_f1|mean\_test_precision|mean\_test_recall|
|---|---|---|---|---|---|
|0|1|0\.7885899814471243|0\.8783597832110669|0\.7849599461238876|0\.9973333333333333|
|1|0\.1|0\.8464543393114822|0\.8900207186160214|0\.9630631044652457|0\.8308421052631578|
|2|0\.01|0\.7915378272521129|0\.8384571396332458|0\.9877403678969146|0\.7363508771929824|
|3|0\.001|0\.8180169037311895|0\.8627812689624499|0\.9820070191164824|0\.776280701754386|
|4|0\.0001|0\.8383426097711812|0\.881035820124733|0\.9753038823036937|0\.8095789473684212|
|5|1e-05|0\.8464852607709752|0\.8898393897152944|0\.9591019619652146|0\.8348947368421052|

##### Oversampled
|index|param\_alpha|mean\_test_accuracy|mean\_test_f1|mean\_test_precision|mean\_test_recall|
|---|---|---|---|---|---|
|0|1|0\.9667549668874171|0\.9654714043194442|0\.9893277893277894|0\.9440701754385964|
|1|0\.1|0\.9787284768211922|0\.9783008516855206|0\.9907714741398952|0\.966719298245614|
|2|0\.01|0\.9734172185430465|0\.972787879198138|0\.9883062883062884|0\.9587368421052631|
|3|0\.001|0\.9694216335540838|0\.9685428172449685|0\.989468066180395|0\.9494210526315789|
|4|0\.0001|0\.9667637969094922|0\.9655854963797628|0\.9893445735550997|0\.9441052631578948|
|5|1e-05|0\.9647637969094923|0\.9633842416790085|0\.9893445735550997|0\.9401052631578949|




In [52]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate

In [93]:
parameters = {'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]}
scorers = {
    'accuracy': make_scorer(accuracy_score),
    'f1': make_scorer(f1_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score)
}
mnb = MultinomialNB()
classifier = GridSearchCV(mnb, parameters, return_train_score=False, cv=10, scoring=scorers, refit='accuracy')
classifier.fit(x_oversampled, y_oversampled)

GridSearchCV(cv=10, estimator=MultinomialNB(),
             param_grid={'alpha': [1, 0.1, 0.01, 0.001, 0.0001, 1e-05]},
             refit='accuracy',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'f1': make_scorer(f1_score),
                      'precision': make_scorer(precision_score),
                      'recall': make_scorer(recall_score)})

In [94]:
results = pd.DataFrame(classifier.cv_results_)
results[['param_alpha', 'mean_test_accuracy', 'mean_test_f1', 'mean_test_precision', 'mean_test_recall']]

,param_alpha,mean_test_accuracy,mean_test_f1,mean_test_precision,mean_test_recall
0,1,0.966755,0.965471,0.989328,0.944070
1,0.1,0.978728,0.978301,0.990771,0.966719
2,0.01,0.973417,0.972788,0.988306,0.958737
3,0.001,0.969422,0.968543,0.989468,0.949421
4,0.0001,0.966764,0.965585,0.989345,0.944105
5,0.00001,0.964764,0.963384,0.989345,0.940105


In [113]:
corpus = pd.read_csv('/content/thesis/Dataset/kaggle_tweets.csv') #read csv

df_shape = corpus.shape # (row, column)
# remove urls, remove special chars, conver to lowercase
for i in range(df_shape[0]):
  string = re.sub(r'http\S+', '', corpus.at[i, 'text']).lower()
  corpus.at[i, 'text'] = re.sub(r'\n', '', string)

stemmer = PorterStemmer() # Porter Stemmer

stopwords_eng = set(stopwords.words('english')) # English stopwords
stopwords_tl  = set(stopwordsiso.stopwords('tl'))
filtered_sentence = []
filtered_sentence2 = []

for i in range(len(corpus['text'])):
  document = corpus.loc[i, 'text']
  tokens = nltk.word_tokenize(document)

  stemmed_tokens = [stemmer.stem(token) for token in tokens] # stem each words
  filtered_sentence = [token for token in stemmed_tokens if not token in stopwords_eng] # remove english stopwords
  filtered_sentence2 = [token for token in filtered_sentence if not token in stopwords_tl] #remove tagalog stopwords

  document = " ".join(filtered_sentence2)
  corpus.loc[i, 'text'] = document

# vectorizer = TfidfVectorizer(ngram_range=(1,4)) # Initialize vectorizer
# x_train_features = vectorizer.fit_transform(x)
x_test_features = vectorizer.transform(corpus['text'])
res = classifier.predict(x_test_features)

In [118]:
for text, sentiment in zip(corpus['text'], res):
  print(sentiment, text)


1 're talk human live ! let 's repeat mistak past ! # dengvaxia jeopard mani lives…
1 first covid-19 vaccin new york lev radin # covidvaccin # biontech # pfizervaccin # pfizerbiontech
1 pleas read thi thread . # covid19 # pfizerbiontech # pfizervaccin # modernavaccin
1 news , @ secduqu rodrigo go # sinovac china , chines get more…
1 see post colleagu us get shot # pfizerbiontech vaccine- thi true…
0 break news : food drug administr ( fda ) philippin approv emerg use author ( eua ) pf…
1 got vaccin . 🙏🏻 # covidvaccin # pfizerbiontech
1 latest count 38 death vaccin # pfizer # covidvaccin # vaccinedeath # germani # pfizerbiontech
1 # sinovac # sinopharm 's covid-19 vaccin veri advanc stage 's review emerg use l…
1 🇨🇳🇮🇩 ha approv # sinovac biotech ’ covid-19 vaccin use elderli , letter food drug age…
1 cambodia plan vaccin least 10 million 16 million popul . # sinovac
1 know differ among vaccin — pfizer , moderna , aztrazeneca , sinopharma , gamaleya , j & amp ; j interview w…
1 health off